This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *

In [3]:
df = printReport\
(
    model='lda',
    splitVersion=2,
    noSubsampling=True,
    metaFilter=\
    {
        # "implementation": "gensim-lda",
    },
    metricsFilter=None, # {'ndcg', 'ndcg@10'},
    sortBy='ndcg',
)

These values are common to all rows:

	- model: lda
	- splitVersion: 2
	- maxDF: 300
	- ldaLearningMethod: online
	- useExtraNews: False
	- maxUsers: None
	- ldaLearningDecay: 0.7
	- lowercase: True


NameError: name 'reorderDFColumns' is not defined

In [ ]:
df = printReport\
(
    model='nmf',
    splitVersion=2,
    noSubsampling=True,
    metaFilter=\
    {
        # "implementation": "gensim-lda",
    },
    metricsFilter=None, # {'ndcg', 'ndcg@10'},
    sortBy='ndcg',
)

In [ ]:
df.columns.get_loc('nbTopics') in df.columns

In [ ]:
def colorise_df_columns(df, grey=None, blue=None, red=None, green=None,
                       grey_rgba=(150, 150, 150, 0.3),
                       green_rgba=(0, 155, 149, 0.3),
                       blue_rgba=(0, 122, 204, 0.3),
                       red_rgba=(242, 82, 82, 0.3)):
    """
        This function return a new DataFrame.
        Choose colors for columns you give in grey, blue, red and green lists.
        
        Usage example:
        
            df = colorise_df_columns(df, grey={'id'}, red={'score', 'loss'})
    """
    # Default are empty sets:
    colors = {'grey': grey, 'blue': blue, 'red': red, 'green': green}
    for color in colors:
        if colors[color] is None:
            colors[color] = set()
    # Making a css_colors dict:
    css_colors = {'grey': grey_rgba, 'blue': blue_rgba, 'red': red_rgba, 'green': green_rgba}
    pre = 'background-color: rgba('
    post = ')'
    for css_color in css_colors:
        css_colors[css_color] = pre + ', '.join([str(e) for e in css_colors[css_color]]) + post
    # Checking if we have duplicates columns:
    union = set()
    total = 0
    for color in colors:
        for column in colors[color]:
            union.add(column)
            total += 1
    if total != len(union):
        raise Exception("Duplicates in columns between colors")
    # Removing elements not in df.columns:
    for color in colors:
        for column in set(colors[color]):
            if column not in df.columns:
                colors[color].remove(column)
    # We define the colorize funct:
    def colorize(df):
        df = pd.DataFrame('', index=df.index, columns=df.columns)
        df.iloc[:, column_index] = current_css_color
        return df
    for color in colors:
        current_css_color = css_colors[color]
        # print(current_css_color)
        for column in colors[color]:
            print(column)
            column_index = df.columns.get_loc(column)
            df = pd.DataFrame(df.style.apply(highlight_col, axis=None))
    return df
colorise_df_columns(df, grey={'A'}, green={'B', 'C'})

In [ ]:
np.random.seed(100)
df =  pd.DataFrame(np.random.randn(5, 3), columns=list('ABC'))
colorise_df_columns(df, red={'A'}, grey={'C'})

In [ ]:
pd.DataFrame(df)

In [ ]:
type(df)